https://pypi.org/project/geopy/

https://ricedh.github.io/03-ner.html



In [1]:
import os
#os.chdir()
os.getcwd()

'c:\\Users\\jlu851\\Downloads'

In [9]:
!pip install geopy
!pip install nltk

You should consider upgrading via the 'C:\Users\jlu851\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\jlu851\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd

In [2]:
import pandas as pd
df_0_20k = pd.read_csv("locIn0-20k.csv")
df_20k_35k = pd.read_csv("locIn20-35k.csv")
df_35_50k = pd.read_csv("locIn35-50k.csv")
vdata = pd.concat([df_0_20k,df_20k_35k,df_35_50k], ignore_index=True, sort=False )

In [4]:
locinfo_df = vdata.dropna()
localInfo = locinfo_df["locationInfo"].tolist()

In [68]:
import geopy
from geopy.geocoders import Nominatim
from geopy.geocoders import get_geocoder_for_service
#get_geocoder_for_service("nominatim")
#geolocator = Nominatim(user_agent="your_app_name")
#geopy.geocoders.options.default_user_agent = "my-application"
#geolocator = Nominatim(user_agent="my-application")
#geolocator = Nominatim()
#location = geolocator.reverse("45.47936,-122.672954")
#type(location.raw)

dict

In [74]:
from geopy.exc import GeocoderTimedOut
import time
def do_geocode(address, attempt=1, max_attempts=15):  ##getting the coordinate of the location data
    try:
        time.sleep(1)
        return geolocator.geocode(address)
    
    except (GeocoderTimedOut, AdapterHTTPError):
        time.sleep(60)
        if attempt <= max_attempts:
            return do_geocode(address, attempt=attempt+1)
        raise


def do_georeverse(cord, attempt=1, max_attempts=15): ##Transform cordinate infomation into country tag
    try:
        time.sleep(1)
        return geolocator.reverse(cord)
    except (GeocoderTimedOut, AdapterHTTPError):
        time.sleep(60)
        if attempt <= max_attempts:
            return do_georeverse(cord, attempt=attempt+1)
        raise


In [70]:
import nltk
print('NTLK Version: %s' % nltk.__version__)
from nltk.tag import StanfordNERTagger
import re
#ner_dir = "/content/gdrive/My Drive/sst2/stanford-ner-2018-02-27/"
ner_dir = os.getcwd() + "\\stanford-ner-2018-02-27\\"
stanford_ner_tagger = StanfordNERTagger(
    ner_dir + 'classifiers/english.all.3class.distsim.crf.ser.gz',
    ner_dir + 'stanford-ner-3.9.1.jar'
)
def extract_place(s):
  loc = []
  x = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",s).split()
  rs = stanford_ner_tagger.tag(x) 
  for r in rs:
    tag_value = r[0]
    tag_type = r[1]
    if tag_type == "LOCATION": ##Find location information from free text description of user file
      loc.append(tag_value)
  return loc

def get_geocode(y):
  loc_info = ",".join(y)
  geo_info = do_geocode(loc_info)
  if geo_info is None:
    y_list = y
    while len(y_list) >= 1:
      loc_info = y[0]
      geo_info = do_geocode(loc_info)
      if geo_info is None:
        del y_list[0]
      else:
        break
  return geo_info


NTLK Version: 3.8.1


In [71]:
def get_location (cord, ind, i):
    try:
      time.sleep(1)
      location = do_georeverse(cord)
    except:
      print("i:", ind)
      print("cords:", cord)
      print("original information:", i)
      return None
    else:
      if location is not None:
        return(location.raw["address"]["country_code"]) ##country code tags
      else:
        return None
      #country_list.append(location.raw["address"]["country_code"])

In [46]:
import time
#from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="sst2-val", timeout = 64000)
country_list= []
res = []
for ind in range(len(localInfo)):
  i = localInfo[ind]
  try:
    geo = do_geocode(i)
    time.sleep(1)
  except:
    geo = None
    #print("failed to request address:", i)
    #res.append(i)
  if geo is not None:
    lat = geo.raw["lat"]
    lon = geo.raw["lon"]
    cord = lat + "," + lon
    loc = get_location(cord, ind, i)
    if loc is not None:
      country_list.append(loc)
    else:
      res.append(ind)
    #try:
    #  location = geolocator.reverse(cord)
    #except:
    #  print("i":, ind)
    #  print("cords:", cord)
    #  print("original information:", i)
    #else:
      #country_list.append(location.raw["address"]["country_code"])
  elif ("ÜT:" in i) or ("iPhone:" in i):
    info = i.split(":")[1]
    loc = get_location(cord, ind, i)
    if loc is not None:
      country_list.append(loc)
    else:
      res.append(ind)
  else:
    place_ner = extract_place(i)
    geo = get_geocode(place_ner)
    time.sleep(1)
    if geo is None:
      print("invalid info:", i, ind)
      res.append(ind)
    if geo is not None:
      lat = geo.raw["lat"]
      lon = geo.raw["lon"]
      cord = lat + "," + lon
      loc = get_location(cord, ind, i)
      if loc is not None:
        country_list.append(loc)
      else:
        res.append(ind)
      #location = geolocator.reverse(cord)
      #country_list.append(location.raw["address"]["country_code"])
  #time.sleep(1)


invalid info: Bong Leach 5
invalid info: Paris/ London/ Milan 26
invalid info: in Hell with Lucifer. 36
invalid info: Columbus, OH/ Memphis, TN 38
invalid info: Sunny Sweden! 45
invalid info: ZZ9 Plural Z Alpha 62
invalid info: Coulson Academy  68
invalid info: Home = Melbs 71
invalid info: a van down by the river 73
invalid info: The Void of Creativity  75
invalid info: Not too far from u 83
invalid info: upyourbutt. 103
invalid info: FashionLand  106
invalid info: Errors?Send@Reply4Review+RT:) 132
invalid info: The Isle of Dr Morehoes 143
invalid info: somewhere on mars 152
invalid info: M I A M I 157
invalid info: ☀️ 196
invalid info: Behind the computer 200
invalid info: En algún Concierto 234
invalid info: https://ko-fi.com/eleidoscope 236
invalid info: Las Vegas, NV & Cornwall, ON 250
invalid info: somewhere IG: @ThisLifesATrip 294
invalid info: Home of the Sonics 304
invalid info: Saraline 325
invalid info: Brooklynite. 329
invalid info: The Happiest Place on Earth 338
invalid i

In [60]:
vdata1 = locinfo_df
vdata1 =vdata1.drop(vdata1.index[res])

In [63]:
vdata1["country"] = country_list ##Adding country code information into the dataframe

In [66]:

vdata1.to_csv("countryInfo0-50k.csv", index = False) 

Country tagging for another file 50k data

In [83]:
import pandas as pd
vdata = pd.read_csv("locIn50-100k.csv")
locinfo_df = vdata.dropna()
localInfo = locinfo_df["locationInfo"].tolist()

In [78]:
import time
#from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="sst2-val", timeout = 64000)
country_list= []
res = []
for ind in range(len(localInfo)):
  i = localInfo[ind]
  try:
    geo = do_geocode(i)
    time.sleep(1)
  except:
    geo = None
    #print("failed to request address:", i)
    #res.append(i)
  if geo is not None:
    lat = geo.raw["lat"]
    lon = geo.raw["lon"]
    cord = lat + "," + lon
    loc = get_location(cord, ind, i)
    if loc is not None:
      country_list.append(loc)
    else:
      res.append(ind)
    #try:
    #  location = geolocator.reverse(cord)
    #except:
    #  print("i":, ind)
    #  print("cords:", cord)
    #  print("original information:", i)
    #else:
      #country_list.append(location.raw["address"]["country_code"])
  elif ("ÜT:" in i) or ("iPhone:" in i):
    info = i.split(":")[1]
    loc = get_location(cord, ind, i)
    if loc is not None:
      country_list.append(loc)
    else:
      res.append(ind)
  else:
    place_ner = extract_place(i)
    geo = get_geocode(place_ner)
    time.sleep(1)
    if geo is None:
      print("invalid info:", i, ind)
      res.append(ind)
    if geo is not None:
      lat = geo.raw["lat"]
      lon = geo.raw["lon"]
      cord = lat + "," + lon
      loc = get_location(cord, ind, i)
      if loc is not None:
        country_list.append(loc)
      else:
        res.append(ind)

invalid info: Jersey732 5
invalid info: In the ticks of Berkshire 15
invalid info: Everywhere all at once. 23
invalid info: Somewhere Positive 24
invalid info: Loading Screen Purgatory  41
invalid info: The Sticks, Texas  46
invalid info: Planet Bullshit 70
invalid info: In My Room Typing This Crap 79
invalid info: The BAY, most likely. 89
invalid info: SA,TX ➡ MI ➡ NC 98
invalid info: American Dreamin'... 114
invalid info: 🇦🇷 120
invalid info: In my own world! 132
invalid info: Under your bed 137
invalid info: my own personal hell 138
invalid info: Earth (or 35,000ft) 148
invalid info: 🇵🇭 160
invalid info: Coast Salish Territories 161
invalid info: ï¿½T: 33.0062382,-96.828722 168
invalid info: Smitten with the Mitten in AZO 179
invalid info: CanaDUH 215
invalid info: your local applebees 216
invalid info: Your fantasies 218
invalid info: HoLLyFANCYwOOD 220
invalid info: Location: Probably by the food 233
invalid info: ☀️🏝🏔 252
invalid info: Between a rock and the casbah 255
invalid i

In [91]:
vdata2 = locinfo_df
vdata2 =vdata2.drop(vdata2.index[res])
vdata2["country"] = country_list
vdata2.to_csv("countryInfo50k-100k.csv", index = False)
print(vdata2.shape)


(18192, 5)
